# Load Training Data

In [1]:
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
input_shape

30

In [4]:
train_names = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
train_column_names = train_names.values.tolist()
train_column_names = [k for row in train_column_names for k in row]

train_data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None)
train_data.columns = train_column_names

### Single dataframe column

y_train = pd.read_csv('../../../data/subject_train.txt', header = None)
y_train.columns = ['Subject']

y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
y_train_activity.columns = ['Activity']

X_train_1 = train_data[sub_features]
X_train_2 = train_data[act_features]
X_train_data = pd.concat([X_train_1, X_train_2], axis = 1)

X_train_data = pd.concat([X_train_data, y_train, y_train_activity], axis = 1)
X_train_data

42 tGravityAcc-mean()-Y  43 tGravityAcc-mean()-Z  \
0                   -0.140840                 0.115375   
1                   -0.141551                 0.109379   
2                   -0.142010                 0.101884   
3                   -0.143976                 0.099850   
4                   -0.148750                 0.094486   
...                       ...                      ...   
7347                -0.222004                -0.039492   
7348                -0.242054                -0.039863   
7349                -0.236950                -0.026805   
7350                -0.233230                -0.004984   
7351                -0.233292                -0.020954   

      51 tGravityAcc-max()-Y  52 tGravityAcc-max()-Z  54 tGravityAcc-min()-Y  \
0                  -0.161265                0.124660               -0.123213   
1                  -0.161343                0.122586               -0.114893   
2                  -0.163711                0.094566               -0.114893   
3                  -0.163711                0.093425               -0.121336   
4                  -0.166786                0.091682               -0.121834   
...                      ...                     ...                     ...   
7347               -0.214233               -0.016391               -0.234998   
7348               -0.231477               -0.016391               -0.234998   
7349               -0.249134                0.024684               -0.216004   
7350               -0.244267                0.024684               -0.210542   
7351               -0.240956                0.003031               -0.212149   

      55 tGravityAcc-min()-Z  56 tGravityAcc-sma()  58 tGravityAcc-energy()-Y  \
0                   0.056483             -0.375426                  -0.970905   
1                   0.102764             -0.383430                  -0.970583   
2                   0.102764             -0.401602                  -0.970368   
3                   0.095753             -0.400278                  -0.969400   
4                   0.094059             -0.400477                  -0.967051   
...                      ...                   ...                        ...   
7347               -0.071977             -0.405132                  -0.918375   
7348               -0.068919             -0.358934                  -0.902880   
7349               -0.068919             -0.377025                  -0.907561   
7350               -0.040009             -0.440050                  -0.910648   
7351               -0.047491             -0.432003                  -0.910579   

      59 tGravityAcc-energy()-Z  475 fBodyGyro-bandsEnergy()-1,8  ...  \
0                     -0.975510                        -0.999454  ...   
1                     -0.978500                        -0.999856  ...   
2                     -0.981672                        -0.999954  ...   
3                     -0.982420                        -0.999931  ...   
4                     -0.984363                        -0.999926  ...   
...                         ...                              ...  ...   
7347                  -0.995193                        -0.053258  ...   
7348                  -0.995151                        -0.029411  ...   
7349                  -0.995450                         0.161404  ...   
7350                  -0.998824                         0.193585  ...   
7351                  -0.998144                        -0.129277  ...   

      272 fBodyAcc-mad()-X  282 fBodyAcc-energy()-X  \
0                -0.996889                -0.999968   
1                -0.997890                -0.999991   
2                -0.994097                -0.999969   
3                -0.994547                -0.999975   
4                -0.997725                -0.999990   
...                    ...                      ...   
7347             -0.050748                -0.674230   
7348             -0.177661                -0.705580   
734

In [5]:
X_train = X_train_data[(X_train_data['Subject'].isin([1, 3, 5, 7, 8, 11, 14, 17, 19])) & (X_train_data['Activity'].isin([1, 3, 4]))].iloc[:,:-2].values
y_train = X_train_data[(X_train_data['Subject'].isin([1, 3, 5, 7, 8, 11, 14, 17, 19])) & (X_train_data['Activity'].isin([1, 3, 4]))].iloc[:,-2].values

In [6]:
for k in range(len(y_train)):
    if y_train[k] == 1:
        y_train[k] = 0
    elif y_train[k] == 3:
        y_train[k] = 1
    elif y_train[k] == 5:
        y_train[k] = 2
    elif y_train[k] == 7:
        y_train[k] = 3
    elif y_train[k] == 8:
        y_train[k] = 4
    elif y_train[k] == 11:
        y_train[k] = 5
    elif y_train[k] == 14:
        y_train[k] = 6
    elif y_train[k] == 17:
        y_train[k] = 7
    else:
        y_train[k] = 8

# Train Test Data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.15, shuffle = True)

In [8]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 20),
            classifier_block(20, 20),
            classifier_block(20, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Classifier()
lr = 0.001
n_epochs = 7500
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [10]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 11.062401056289673, Final Batch Loss: 2.206573009490967
Epoch 2, Loss: 11.047949314117432, Final Batch Loss: 2.20697283744812
Epoch 3, Loss: 11.031243562698364, Final Batch Loss: 2.213114023208618
Epoch 4, Loss: 11.01078987121582, Final Batch Loss: 2.1969521045684814
Epoch 5, Loss: 10.995674133300781, Final Batch Loss: 2.1987383365631104
Epoch 6, Loss: 10.975208520889282, Final Batch Loss: 2.194032669067383
Epoch 7, Loss: 10.942647695541382, Final Batch Loss: 2.1809425354003906
Epoch 8, Loss: 10.914919137954712, Final Batch Loss: 2.1807444095611572
Epoch 9, Loss: 10.878669261932373, Final Batch Loss: 2.1838366985321045
Epoch 10, Loss: 10.84848666191101, Final Batch Loss: 2.163916826248169
Epoch 11, Loss: 10.804754495620728, Final Batch Loss: 2.1554720401763916
Epoch 12, Loss: 10.783970355987549, Final Batch Loss: 2.1681947708129883
Epoch 13, Loss: 10.701191186904907, Final Batch Loss: 2.124854564666748
Epoch 14, Loss: 10.644819498062134, Final Batch Loss: 2.1391673088073

Epoch 120, Loss: 6.510164141654968, Final Batch Loss: 1.2849467992782593
Epoch 121, Loss: 6.531868815422058, Final Batch Loss: 1.4228031635284424
Epoch 122, Loss: 6.40522563457489, Final Batch Loss: 1.2522951364517212
Epoch 123, Loss: 6.377614617347717, Final Batch Loss: 1.3634977340698242
Epoch 124, Loss: 6.299414873123169, Final Batch Loss: 1.2905850410461426
Epoch 125, Loss: 6.339807987213135, Final Batch Loss: 1.2545335292816162
Epoch 126, Loss: 6.367006778717041, Final Batch Loss: 1.231135606765747
Epoch 127, Loss: 6.168930411338806, Final Batch Loss: 1.198753833770752
Epoch 128, Loss: 6.301113963127136, Final Batch Loss: 1.2911702394485474
Epoch 129, Loss: 6.331954836845398, Final Batch Loss: 1.2811098098754883
Epoch 130, Loss: 6.242249011993408, Final Batch Loss: 1.2198376655578613
Epoch 131, Loss: 6.228593111038208, Final Batch Loss: 1.2517372369766235
Epoch 132, Loss: 6.157120704650879, Final Batch Loss: 1.3363173007965088
Epoch 133, Loss: 6.05823278427124, Final Batch Loss: 1

Epoch 233, Loss: 5.233388245105743, Final Batch Loss: 1.0708333253860474
Epoch 234, Loss: 5.409892678260803, Final Batch Loss: 1.1055485010147095
Epoch 235, Loss: 5.148668050765991, Final Batch Loss: 1.1084096431732178
Epoch 236, Loss: 5.2527337074279785, Final Batch Loss: 1.0462441444396973
Epoch 237, Loss: 5.285170555114746, Final Batch Loss: 0.9205756187438965
Epoch 238, Loss: 5.193244278430939, Final Batch Loss: 1.0352177619934082
Epoch 239, Loss: 5.1697341203689575, Final Batch Loss: 0.9533716440200806
Epoch 240, Loss: 5.315905451774597, Final Batch Loss: 1.0744231939315796
Epoch 241, Loss: 5.213665008544922, Final Batch Loss: 1.0120534896850586
Epoch 242, Loss: 5.12600439786911, Final Batch Loss: 0.9205324649810791
Epoch 243, Loss: 5.084780156612396, Final Batch Loss: 1.0538547039031982
Epoch 244, Loss: 5.13839066028595, Final Batch Loss: 1.0994629859924316
Epoch 245, Loss: 5.115194618701935, Final Batch Loss: 0.9699789881706238
Epoch 246, Loss: 5.179529905319214, Final Batch Los

Epoch 347, Loss: 4.6550344824790955, Final Batch Loss: 0.9377731084823608
Epoch 348, Loss: 4.766660928726196, Final Batch Loss: 0.923858642578125
Epoch 349, Loss: 4.642500400543213, Final Batch Loss: 0.8751749396324158
Epoch 350, Loss: 4.588007748126984, Final Batch Loss: 0.9333604574203491
Epoch 351, Loss: 4.627453148365021, Final Batch Loss: 1.0226633548736572
Epoch 352, Loss: 4.832939505577087, Final Batch Loss: 0.9493738412857056
Epoch 353, Loss: 4.666880309581757, Final Batch Loss: 0.9508885741233826
Epoch 354, Loss: 4.680314540863037, Final Batch Loss: 0.8849192261695862
Epoch 355, Loss: 4.549729347229004, Final Batch Loss: 0.8895740509033203
Epoch 356, Loss: 4.748805582523346, Final Batch Loss: 1.031207799911499
Epoch 357, Loss: 4.7753530740737915, Final Batch Loss: 0.9393377304077148
Epoch 358, Loss: 4.679135262966156, Final Batch Loss: 0.9126588106155396
Epoch 359, Loss: 4.6729336977005005, Final Batch Loss: 0.9123710989952087
Epoch 360, Loss: 4.583589851856232, Final Batch Lo

Epoch 465, Loss: 4.348120748996735, Final Batch Loss: 0.8001621961593628
Epoch 466, Loss: 4.333989322185516, Final Batch Loss: 0.8978897333145142
Epoch 467, Loss: 4.279716193675995, Final Batch Loss: 0.9478644728660583
Epoch 468, Loss: 4.540060520172119, Final Batch Loss: 0.8721200823783875
Epoch 469, Loss: 4.373999893665314, Final Batch Loss: 0.823943555355072
Epoch 470, Loss: 4.2347413301467896, Final Batch Loss: 0.8879616856575012
Epoch 471, Loss: 4.237214088439941, Final Batch Loss: 0.7355800271034241
Epoch 472, Loss: 4.26641446352005, Final Batch Loss: 0.897770345211029
Epoch 473, Loss: 4.3121243715286255, Final Batch Loss: 0.8725606799125671
Epoch 474, Loss: 4.248126268386841, Final Batch Loss: 0.8842424750328064
Epoch 475, Loss: 4.275539755821228, Final Batch Loss: 0.8620871901512146
Epoch 476, Loss: 4.242079377174377, Final Batch Loss: 0.9529075026512146
Epoch 477, Loss: 4.158293068408966, Final Batch Loss: 0.8300679922103882
Epoch 478, Loss: 4.283039331436157, Final Batch Loss

...

Epoch 6985, Loss: 1.4862850904464722, Final Batch Loss: 0.27653589844703674
Epoch 6986, Loss: 1.545218139886856, Final Batch Loss: 0.250889390707016
Epoch 6987, Loss: 1.842942714691162, Final Batch Loss: 0.5221807360649109
Epoch 6988, Loss: 1.6658715903759003, Final Batch Loss: 0.3468872606754303
Epoch 6989, Loss: 1.5920735895633698, Final Batch Loss: 0.30974671244621277
Epoch 6990, Loss: 1.726595789194107, Final Batch Loss: 0.2596084773540497
Epoch 6991, Loss: 1.65511953830719, Final Batch Loss: 0.32098713517189026
Epoch 6992, Loss: 1.6266534924507141, Final Batch Loss: 0.31876620650291443
Epoch 6993, Loss: 1.6357618570327759, Final Batch Loss: 0.39411988854408264
Epoch 6994, Loss: 1.6959164440631866, Final Batch Loss: 0.3002450466156006
Epoch 6995, Loss: 1.6770785748958588, Final Batch Loss: 0.38182199001312256
Epoch 6996, Loss: 1.6038898527622223, Final Batch Loss: 0.31106817722320557
Epoch 6997, Loss: 1.7933639287948608, Final Batch Loss: 0.4241999685764313
Epoch 6998, Loss: 1.7240

Epoch 7094, Loss: 1.6233860552310944, Final Batch Loss: 0.2902008295059204
Epoch 7095, Loss: 1.7352189123630524, Final Batch Loss: 0.3774230480194092
Epoch 7096, Loss: 1.713057041168213, Final Batch Loss: 0.3264218270778656
Epoch 7097, Loss: 1.6714014112949371, Final Batch Loss: 0.3465871810913086
Epoch 7098, Loss: 1.5866968631744385, Final Batch Loss: 0.296328067779541
Epoch 7099, Loss: 1.5355514734983444, Final Batch Loss: 0.3755209445953369
Epoch 7100, Loss: 1.6131525039672852, Final Batch Loss: 0.2772896885871887
Epoch 7101, Loss: 1.582241028547287, Final Batch Loss: 0.2998253107070923
Epoch 7102, Loss: 1.7089681029319763, Final Batch Loss: 0.3208198845386505
Epoch 7103, Loss: 1.5802728235721588, Final Batch Loss: 0.32834094762802124
Epoch 7104, Loss: 1.697683870792389, Final Batch Loss: 0.34229451417922974
Epoch 7105, Loss: 1.6481459438800812, Final Batch Loss: 0.3274984359741211
Epoch 7106, Loss: 1.7004454135894775, Final Batch Loss: 0.31681764125823975
Epoch 7107, Loss: 1.728330

Epoch 7210, Loss: 1.7940165400505066, Final Batch Loss: 0.3204159140586853
Epoch 7211, Loss: 1.8099472224712372, Final Batch Loss: 0.33623242378234863
Epoch 7212, Loss: 1.4524375200271606, Final Batch Loss: 0.2976800799369812
Epoch 7213, Loss: 1.5182016491889954, Final Batch Loss: 0.28702548146247864
Epoch 7214, Loss: 1.6344954669475555, Final Batch Loss: 0.35592901706695557
Epoch 7215, Loss: 1.6327647268772125, Final Batch Loss: 0.35395076870918274
Epoch 7216, Loss: 1.5695189833641052, Final Batch Loss: 0.30261099338531494
Epoch 7217, Loss: 1.6409631073474884, Final Batch Loss: 0.3381268084049225
Epoch 7218, Loss: 1.6660723090171814, Final Batch Loss: 0.3285170793533325
Epoch 7219, Loss: 1.4476156681776047, Final Batch Loss: 0.2475397139787674
Epoch 7220, Loss: 1.4957085251808167, Final Batch Loss: 0.27686819434165955
Epoch 7221, Loss: 1.773669809103012, Final Batch Loss: 0.3594565689563751
Epoch 7222, Loss: 1.3860109895467758, Final Batch Loss: 0.30874431133270264
Epoch 7223, Loss: 1

Epoch 7323, Loss: 1.8267613351345062, Final Batch Loss: 0.5001497864723206
Epoch 7324, Loss: 1.7259505093097687, Final Batch Loss: 0.2619786858558655
Epoch 7325, Loss: 1.7815261781215668, Final Batch Loss: 0.48257169127464294
Epoch 7326, Loss: 1.619974672794342, Final Batch Loss: 0.31615355610847473
Epoch 7327, Loss: 1.6266626715660095, Final Batch Loss: 0.38093769550323486
Epoch 7328, Loss: 1.5539408028125763, Final Batch Loss: 0.35920751094818115
Epoch 7329, Loss: 1.5812170207500458, Final Batch Loss: 0.3081994354724884
Epoch 7330, Loss: 1.7661359310150146, Final Batch Loss: 0.3280143439769745
Epoch 7331, Loss: 1.6641694009304047, Final Batch Loss: 0.3918263912200928
Epoch 7332, Loss: 1.5665457844734192, Final Batch Loss: 0.3412179946899414
Epoch 7333, Loss: 1.517937794327736, Final Batch Loss: 0.2443837970495224
Epoch 7334, Loss: 1.8445504307746887, Final Batch Loss: 0.27358731627464294
Epoch 7335, Loss: 1.684929370880127, Final Batch Loss: 0.35028064250946045
Epoch 7336, Loss: 1.59

Epoch 7434, Loss: 1.6140491962432861, Final Batch Loss: 0.3117530643939972
Epoch 7435, Loss: 1.4524189233779907, Final Batch Loss: 0.32434964179992676
Epoch 7436, Loss: 1.6977427005767822, Final Batch Loss: 0.31840136647224426
Epoch 7437, Loss: 1.6013046503067017, Final Batch Loss: 0.36407724022865295
Epoch 7438, Loss: 1.7006056308746338, Final Batch Loss: 0.3297506868839264
Epoch 7439, Loss: 1.8012837171554565, Final Batch Loss: 0.3053029179573059
Epoch 7440, Loss: 1.5385586023330688, Final Batch Loss: 0.3189316391944885
Epoch 7441, Loss: 1.4964328110218048, Final Batch Loss: 0.3111027181148529
Epoch 7442, Loss: 1.4834253191947937, Final Batch Loss: 0.2800998389720917
Epoch 7443, Loss: 1.5778461694717407, Final Batch Loss: 0.3435657322406769
Epoch 7444, Loss: 1.5158578753471375, Final Batch Loss: 0.3303665518760681
Epoch 7445, Loss: 1.5181241184473038, Final Batch Loss: 0.2329643815755844
Epoch 7446, Loss: 1.6185244619846344, Final Batch Loss: 0.3988698422908783
Epoch 7447, Loss: 1.80

In [11]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 3))

[[23  0  0  0  1  0  0  0  1]
 [ 0 19  0  0  1  2  0  4  0]
 [ 0  0 19  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0]
 [ 2  1  0  0 19  0  0  0  1]
 [ 1  0  0  0  0 25  0  0  1]
 [ 0  0  0  5  0  0 23  0  0]
 [ 0  1  0  0  0  0  0 27  0]
 [ 0  0  0  0  0  1  0  0 18]]
              precision    recall  f1-score   support

           0      0.885     0.920     0.902        25
           1      0.905     0.731     0.809        26
           2      1.000     1.000     1.000        19
           3      0.800     1.000     0.889        20
           4      0.905     0.826     0.864        23
           5      0.893     0.926     0.909        27
           6      1.000     0.821     0.902        28
           7      0.871     0.964     0.915        28
           8      0.857     0.947     0.900        19

    accuracy                          0.898       215
   macro avg      0.902     0.904     0.899       215
weighted avg      0.903     0.898     0.896       215



In [12]:
torch.save(model.state_dict(), f'../../../saved_models/UCI 9 User Classifier Ablation')